# Model comparison

In [ ]:
import os
import sys
import pandas as pd
import re
import numpy as np
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Define the root directory
root = Path.cwd().parents[1]
results_root = root / "results/model_reports/"

# Function to extract data
def extract_metrics(section):
    metrics = {}
    accuracy = section[1]
    report = section[2]

    # Extract accuracy
    metrics['Accuracy'] = float(accuracy)
    
    # Extract individual class metrics
    class_metrics = re.findall(r'(\d.\d): \{\'precision\': ([\d.]+), \'recall\': ([\d.]+), \'f1-score\': ([\d.]+), \'support\': ([\d.]+)\}', report)
    for metric in class_metrics:
        class_id = metric[0]
        metrics[f'Class_{class_id}_precision'] = float(metric[1])
        metrics[f'Class_{class_id}_recall'] = float(metric[2])
        metrics[f'Class_{class_id}_f1-score'] = float(metric[3])
        metrics[f'Class_{class_id}_support'] = float(metric[4])
    
    # Extract macro avg metrics
    macro_avg = re.search(r'macro avg: \{\'precision\': ([\d.]+), \'recall\': ([\d.]+), \'f1-score\': ([\d.]+), \'support\': ([\d.]+)\}', report)
    metrics['macro_avg_precision'] = float(macro_avg.group(1))
    metrics['macro_avg_recall'] = float(macro_avg.group(2))
    metrics['macro_avg_f1-score'] = float(macro_avg.group(3))
    metrics['macro_avg_support'] = float(macro_avg.group(4))
    
    # Extract weighted avg metrics
    weighted_avg = re.search(r'weighted avg: \{\'precision\': ([\d.]+), \'recall\': ([\d.]+), \'f1-score\': ([\d.]+), \'support\': ([\d.]+)\}', report)
    metrics['weighted_avg_precision'] = float(weighted_avg.group(1))
    metrics['weighted_avg_recall'] = float(weighted_avg.group(2))
    metrics['weighted_avg_f1-score'] = float(weighted_avg.group(3))
    metrics['weighted_avg_support'] = float(weighted_avg.group(4))
    
    return metrics

# List to store all data
all_data = []

In [ ]:
# Iterate over all directories in the results/model_reports directory
for results in results_root.glob("*/"):
    # Extract the directory name
    directory_name = results.name
    
    # Iterate over all txt files in the directory
    for txt_file in results.glob("*.txt"):
        # Read the text file
        with open(txt_file, 'r') as file:
            data = file.read()
        
        # Extract the model name from the file path
        model_name = txt_file.stem.replace('_results', '')

        # Generalize the regular expression to split the data into sections
        sections = re.split(r'(\w+) - (\w+) - Accuracy: ([\d.]+)', data)[1:]

        # Process each section and store the results in a list
        for i in range(0, len(sections), 4):
            classifier_name = sections[i]
            split_type = sections[i+1]
            accuracy = sections[i+2]
            section_data = sections[i+3]
            section_metrics = extract_metrics([split_type, accuracy, section_data])
            section_metrics['Split'] = split_type
            section_metrics['Model'] = model_name
            section_metrics['Directory'] = directory_name
            all_data.append(section_metrics)

# Create DataFrame
combined_df = pd.DataFrame(all_data)

# Ensure 'Directory' is the first column, 'Model' is the second column, and 'Split' is the third column
columns = ['Directory', 'Model', 'Split'] + [col for col in combined_df.columns if col not in ['Directory', 'Model', 'Split']]
combined_df = combined_df[columns]
combined_df


for directory in combined_df['Directory'].unique():
    # Filter the DataFrame for the current directory
    subset_df = combined_df[combined_df['Directory'] == directory]
    
    # Sort the DataFrame by 'Accuracy' for the first plot
    sorted_df_accuracy = subset_df.sort_values(by='Accuracy', ascending=False)
    
    # Sort the DataFrame by 'weighted_avg_f1-score' for the second plot
    sorted_df_f1 = subset_df.sort_values(by='weighted_avg_f1-score', ascending=False)
    
    # Create subplots with 2 columns
    fig, axes = plt.subplots(1, 2, figsize=(20, 6))
    
    # Create the scatter plot for Accuracy
    sns.scatterplot(ax=axes[0], data=sorted_df_accuracy, 
                    y='Model', 
                    x='Accuracy', 
                    marker='s', 
                    hue='Split', palette=['darkorange', 'grey', 'darkred'])
    
    # Set plot title for Accuracy
    axes[0].set_title(f'Performance Metrics (Accuracy) for {directory}')
    
    # Create the scatter plot for weighted_avg_f1-score
    sns.scatterplot(ax=axes[1], data=sorted_df_f1, 
                    y='Model', 
                    x='weighted_avg_f1-score', 
                    marker='s', 
                    hue='Split', palette=['darkorange', 'grey', 'darkred'])
    
    # Set plot title for weighted_avg_f1-score
    axes[1].set_title(f'Performance Metrics (Weighted Avg F1-Score) for {directory}')
    
    # Adjust layout
    plt.tight_layout()
    
    # Show the plot
    plt.show()


